In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pickle
from glob import glob
import re
from concurrent.futures import ProcessPoolExecutor, as_completed

import numpy as np
import pandas as pd
#from tqdm import tqdm
from scipy import stats
from sklearn.metrics import pairwise_distances

import utils.constants as constants
# from src.data import PhenoInfo, PhenoResults, get_all_tissues, get_genes
from data.multixcan_data import MXPhenoInfo, MXPhenoResults
from utils.utils import is_number, chunker

In [3]:
genes_associations_dir = os.path.join(constants.PREPROCESSED_BASED_DIR, 'gene_associations')
smultixcan_gene_association_dirs = os.path.join(genes_associations_dir, 'mashr')

# Load metadata

In [4]:
with open(os.path.join(constants.PREPROCESSED_METADATA_DIR, 'genes_mapping_simplified-0.pkl'), 'rb') as f:
    genes_mapping_0 = pickle.load(f)

with open(os.path.join(constants.PREPROCESSED_METADATA_DIR, 'genes_mapping_simplified-1.pkl'), 'rb') as f:
    genes_mapping_1 = pickle.load(f)

# Load MultiXcan associations

In [5]:
genes_associations_filename = os.path.join(smultixcan_gene_association_dirs, 'smultixcan-genes_associations-pvalue.pkl.xz')
display(genes_associations_filename)

genes_associations = pd.read_pickle(genes_associations_filename)

'/mnt/phenomexcan/results/preprocessed_data/gene_associations/mashr/smultixcan-genes_associations-pvalue.pkl.xz'

In [6]:
# replace inf zscores (from pvalues with 0.0) by a value of 40
genes_associations = genes_associations.replace(np.inf, 40)

assert not genes_associations.isin([np.inf, -np.inf]).any().any()

In [7]:
assert genes_associations.isna().sum().sum() == 0

In [8]:
display(genes_associations.shape)
display(genes_associations.head())

(22255, 4083)

,L12_EPIDERMALTHICKOTH-Other_epidermal_thickening,O42-Diagnoses_main_ICD10_O42_Premature_rupture_of_membranes,20002_1077-Noncancer_illness_code_selfreported_heart_arrhythmia,20445-Depression_possibly_related_to_childbirth,20077-Number_of_diet_questionnaires_completed,22601_91392832-Job_coding_other_work_in_this_industry_factory_hand_mate_assistant_handler_loader,I9_VTE-Venous_thromboembolism,22617_1161-Job_SOC_coding_Transport_and_distribution_managers,20002_1460-Noncancer_illness_code_selfreported_rectal_or_colon_adenomapolyps,5181-Ever_had_eye_surgery,...,20090_394-Type_of_fatoil_used_in_cooking_Unknown_soft_margarine,22617_3512-Job_SOC_coding_Aircraft_pilots_and_flight_engineers,6034-Target_heart_rate_achieved,20003_1140883066-Treatmentmedication_code_insulin_product,22601_41223241-Job_coding_accounts_and_wages_clerkassistantsupervisor_bookkeeper_cost_or_ledger_clerk_audit_assistant_budget_officer_student_loans_officer_paymaster,I82-Diagnoses_main_ICD10_I82_Other_venous_embolism_and_thrombosis,20107_12-Illnesses_of_father_Severe_depression,B07-Diagnoses_main_ICD10_B07_Viral_warts,22601_12253140-Job_coding_sports_centre_manager_riding_school_owner_sports_ground_manager_baths_manager,2664_2-Reason_for_reducing_amount_of_alcohol_drunk_Doctors_advice
gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000000419,0.905638,0.235957,0.612720,0.394017,0.545799,0.310141,0.381676,0.045999,0.219945,0.016343,...,0.341881,0.441179,0.430497,0.346232,0.141478,0.506720,0.787062,0.985367,0.038634,0.488172
ENSG00000000457,0.602945,0.287990,0.191306,0.322591,0.047495,0.696575,0.119221,0.434819,0.559861,0.968186,...,0.916819,0.634916,0.995800,0.077353,0.993128,0.589764,0.180067,0.456236,0.274874,0.910102
ENSG00000000460,0.855718,0.649149,0.672484,0.831449,0.262071,0.552346,0.000613,0.407571,0.380482,0.305636,...,0.855875,0.538767,0.215868,0.831682,0.307241,0.765236,0.119641,0.657201,0.793538,0.952102
ENSG00000000938,0.772473,0.831685,0.241606,0.462259,0.940733,0.530891,0.015450,0.006593,0.512226,0.591307,...,0.771783,0.473211,0.712487,0.134422,0.963751,0.985013,0.215447,0.380525,0.499610,0.001517
ENSG00000000971,0.641797,0.293198,0.425210,0.701933,0.295695,0.141537,0.151129,0.755468,0.224962,0.158145,...,0.090722,0.000368,0.041335,0.778288,0.350304,0.347178,0.885232,0.551847,0.999753,0.680476


# Load PheWAS catalog

In [9]:
phewas_catalog = pd.read_csv(os.path.join(constants.DATA_DIR, 'phewas-catalog.csv'), dtype={'phewas code': str})

In [10]:
phewas_catalog.shape

(215107, 9)

In [11]:
phewas_catalog[phewas_catalog['phewas code'].isna()].head()

,chromosome,snp,phewas phenotype,cases,p-value,odds-ratio,gene_name,phewas code,gwas-associations


In [12]:
phewas_catalog[phewas_catalog['gene_name'].isna()].head()

,chromosome,snp,phewas phenotype,cases,p-value,odds-ratio,gene_name,phewas code,gwas-associations
41,4 111710169,rs2200733,Atrial fibrillation,1950,1.527000e-10,1.5170,NaN,427.21,"Atrial fibrillation, Atrial fibrillation/atria..."
49,4 111710169,rs2200733,Atrial fibrillation & flutter,2041,1.019000e-09,1.4810,NaN,427.2,"Atrial fibrillation, Atrial fibrillation/atria..."
98,4,rs4698036,Gout,769,7.803000e-08,0.6839,NaN,274.1,Serum uric acid
108,4,rs4698036,Gout and other crystal arthropathies,904,1.990000e-07,0.7132,NaN,274,Serum uric acid
115,8 128485038,rs1447295,Prostate cancer,848,2.758000e-07,1.6060,NaN,185,Prostate cancer


In [13]:
phewas_catalog[phewas_catalog['gene_name'].isna()].shape

(52140, 9)

In [14]:
phewas_catalog = phewas_catalog.dropna(subset=['gene_name', 'phewas code'])

In [15]:
phewas_catalog.shape

(162967, 9)

In [16]:
phewas_catalog['gene_name'].unique().shape

(1775,)

In [17]:
phewas_catalog['phewas code'].unique().shape

(1358,)

In [18]:
phewas_catalog = phewas_catalog.assign(gene_id=phewas_catalog['gene_name'].apply(lambda x: genes_mapping_1[x] if x in genes_mapping_1 else None))

In [19]:
phewas_catalog = phewas_catalog.dropna(subset=['gene_name', 'gene_id', 'phewas code'])

In [20]:
phewas_catalog.shape

(147970, 10)

In [21]:
phewas_catalog.head()

,chromosome,snp,phewas phenotype,cases,p-value,odds-ratio,gene_name,phewas code,gwas-associations,gene_id
0,19 45395619,rs2075650,Alzheimer's disease,737,5.237000e-28,2.410,TOMM40,290.11,"Alzheimer's disease, Alzheimer's disease bioma...",ENSG00000130204
1,19 45395619,rs2075650,Dementias,1170,2.409000e-26,2.114,TOMM40,290.1,"Alzheimer's disease, Alzheimer's disease bioma...",ENSG00000130204
2,6 396321,rs12203592,Actinic keratosis,2505,4.141000e-26,1.691,IRF4,702.1,"Eye color, Hair color, Freckling, Progressive ...",ENSG00000137265
3,6 26093141,rs1800562,Iron metabolism disorder,40,3.409000e-25,12.270,HFE,275.1,"Mean corpuscular hemoglobin, Glycated hemoglob...",ENSG00000010704
4,19 45395619,rs2075650,Delirium dementia and amnestic disorders,1566,8.027000e-24,1.841,TOMM40,290,"Alzheimer's disease, Alzheimer's disease bioma...",ENSG00000130204


In [22]:
phewas_catalog.sort_values('phewas phenotype').head()

,chromosome,snp,phewas phenotype,cases,p-value,odds-ratio,gene_name,phewas code,gwas-associations,gene_id
35306,10,rs7923609,ASCVD,166,0.008094,1.3610,JMJD1C,414.2,Alkaline phosphatase,ENSG00000171988
154790,22,rs1012068,ASCVD,166,0.035970,1.2920,DEPDC5,414.2,Chronic Hepatitis C infection,ENSG00000100150
72358,5 158814533,rs10045431,ASCVD,166,0.016740,0.7242,IL12B,414.2,Crohn's disease,ENSG00000113302
130720,14 87896435,rs17124581,ASCVD,166,0.030370,1.6090,SPATA7,414.2,Cognitive performance,ENSG00000042317
184453,6 31912648,rs429608,ASCVD,166,0.042840,1.3440,SKIV2L,414.2,Age-related macular degeneration,ENSG00000204351


# Genes in common

In [23]:
shared_gene_ids = \
    set(phewas_catalog['gene_id'].values)\
    .intersection(genes_associations.index)

In [24]:
len(shared_gene_ids)

1589

# HPO to MIM

In [25]:
hpo_to_mim = pd.read_csv(os.path.join(constants.DATA_DIR, 'hpo-to-omim-and-phecode.csv'), dtype={'phecode': str})

In [26]:
hpo_to_mim.shape

(84031, 10)

In [27]:
hpo_to_mim.head()

,term_id,name,match_available,phecode,phecode string,match_type,class,dID,disease_name,modifier
0,28,Cryptorchidism,1,751.12,Congenital anomalies of male genital organs,General,Congenital,100050,"100050 AARSKOG SYNDROME, AUTOSOMAL DOMINANT",O
1,49,Shawl scrotum,1,751.12,Congenital anomalies of male genital organs,General,Congenital,100050,"100050 AARSKOG SYNDROME, AUTOSOMAL DOMINANT",O
2,175,Cleft palate,1,749.1,Cleft palate,Exact,Congenital,100050,"100050 AARSKOG SYNDROME, AUTOSOMAL DOMINANT",O
3,202,Oral cleft,1,749.1,Cleft palate,Broader,Congenital,100050,"100050 AARSKOG SYNDROME, AUTOSOMAL DOMINANT",O
4,204,Cleft upper lip,1,749.1,Cleft palate,Broader,Congenital,100050,"100050 AARSKOG SYNDROME, AUTOSOMAL DOMINANT",O


# Load silver standard to map from UKB to MIM

In [28]:
omim_silver_standard = pd.read_csv(os.path.join(constants.DATA_DIR, 'omim_silver_standard.tsv'), sep='\t')

In [29]:
ukb_to_mim_map = omim_silver_standard[['trait', 'pheno_mim']].dropna()

In [30]:
ukb_to_mim_map.shape

(7822, 2)

In [31]:
ukb_to_mim_map.head()

,trait,pheno_mim
0,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800
1,M41-Diagnoses_main_ICD10_M41_Scoliosis,102500
2,M41-Diagnoses_main_ICD10_M41_Scoliosis,105830
3,M41-Diagnoses_main_ICD10_M41_Scoliosis,108120
4,M41-Diagnoses_main_ICD10_M41_Scoliosis,108145


# Read gwas2gene (Yanyu) results

In [32]:
from glob import glob

import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [33]:
gwas2gene_results_dir = '/mnt/phenomexcan/results/roc_validation/ukb_gwas2gene_results_omim_silver_standard/'

In [34]:
readRDS = robjects.r['readRDS']

In [35]:
f_files = glob(os.path.join(gwas2gene_results_dir, '*.rds'))
display(len(f_files))

if len(f_files) != len(omim_silver_standard['trait'].unique()):
    print(f'WARNING: some files are not there. {len(omim_silver_standard.trait.unique())} expected, {len(f_files)} found.')

99

In [36]:
gwas2genes_results = {}

for f in f_files:
    f_base = os.path.basename(f)
    f_code = f_base.split('.')[0]
    
    #print(f_base)
    rds_contents = readRDS(f)
    
    if len(rds_contents[1]) > 0:
        f_gene_list = list(rds_contents[1][0].iter_labels())
    else:
        print(f'{f_code}: empty')
        f_gene_list = []
    
    gwas2genes_results[f_code] = f_gene_list

In [37]:
gwas2gene_all_genes = []

for k in gwas2genes_results.keys():
    gwas2gene_all_genes.extend(gwas2genes_results[k])

display(len(gwas2gene_all_genes))

gwas2gene_all_genes = set(gwas2gene_all_genes)
display(len(gwas2gene_all_genes))

gwas2gene_all_genes = shared_gene_ids.intersection(gwas2gene_all_genes)
display(len(gwas2gene_all_genes))

20837

10185

956

In [38]:
pd.Series(list(gwas2gene_all_genes)).head()

0    ENSG00000182348
1    ENSG00000160856
2    ENSG00000143119
3    ENSG00000172575
4    ENSG00000140945
dtype: object

# Merge

In [43]:
# mim to phecode
_tmp = pd.merge(ukb_to_mim_map, hpo_to_mim[['phecode', 'dID']], left_on='pheno_mim', right_on='dID', how='inner').drop(columns=['dID'])
display(_tmp.shape)

(154338, 3)

In [44]:
# phecode to phewas catalog
_tmp = pd.merge(_tmp, phewas_catalog[['phewas code', 'gene_name', 'gene_id']], left_on='phecode', right_on='phewas code', how='inner').drop(columns=['phewas code'])
display(_tmp.shape)

(10256175, 5)

In [45]:
_tmp.head()

,trait,pheno_mim,phecode,gene_name,gene_id
0,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,257.1,UMOD,ENSG00000169344
1,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,257.1,UMOD,ENSG00000169344
2,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,257.1,GLG1,ENSG00000090863
3,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,257.1,STIM2,ENSG00000109689
4,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,257.1,MLPH,ENSG00000115648


In [47]:
# just wondering if this is the right way... but I don't think so, I want just a ukb-trait to gene map and see how many are significant
#_tmp.drop_duplicates(subset=['trait', 'phecode', 'gene_id']).shape

In [48]:
_tmp = _tmp.drop_duplicates(subset=['trait', 'gene_id'])

In [49]:
_tmp.shape

(153579, 5)

In [51]:
from clustering.biclustering.analysis import Trait

In [52]:
_tmp = _tmp.assign(ukb_code=[Trait(t).trait_code for t in _tmp['trait'].values])

In [53]:
_tmp.head()

,trait,pheno_mim,phecode,gene_name,gene_id,ukb_code
0,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,257.1,UMOD,ENSG00000169344,M41
2,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,257.1,GLG1,ENSG00000090863,M41
3,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,257.1,STIM2,ENSG00000109689,M41
4,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,257.1,MLPH,ENSG00000115648,M41
5,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,257.1,JMJD1C,ENSG00000171988,M41


In [54]:
phewas_ukb_results = _tmp

### Keep genes per trait that we can find in GWAS

In [55]:
trait_genes_to_keep = []

# idx = 0
for trait, data in phewas_ukb_results.groupby(['trait']):
    ukb_trait_code = data.ukb_code.unique()[0]
    
    if ukb_trait_code in gwas2genes_results.keys():
        ukb_traits_genes = gwas2genes_results[ukb_trait_code]
    else:
        print(f'WARNING: for ukb trait "{trait}", no genes were found')
        continue
    
    ukb_traits_genes = set(ukb_traits_genes)
    ukb_traits_genes = ukb_traits_genes.intersection(gwas2gene_all_genes)
    
    #genes_intersection = set(omim_silver_standard['ensembl_gene_id'].values).intersection(ukb_traits_genes)
    
    idx_to_keep = data[data['gene_id'].isin(ukb_traits_genes)].index.tolist()
    trait_genes_to_keep.extend(idx_to_keep)
    
#     traits_codes_genes_tuples = [(f'{trait}-{mim}', g) for mim in data['pheno_mim'].unique() for g in ukb_traits_genes]
#     trait_genes_to_keep.extend(traits_codes_genes_tuples)

In [56]:
display(len(trait_genes_to_keep))

2204

In [57]:
display(trait_genes_to_keep[:5])

[319791, 319794, 3342736, 3342786, 3444215]

In [58]:
phewas_ukb_results = phewas_ukb_results.loc[trait_genes_to_keep]

In [59]:
phewas_ukb_results.head()

,trait,pheno_mim,phecode,gene_name,gene_id,ukb_code
319791,1200-Sleeplessness_insomnia,168605,378,IKZF4,ENSG00000123411,1200
319794,1200-Sleeplessness_insomnia,168605,378,PRIM1,ENSG00000198056,1200
3342736,1200-Sleeplessness_insomnia,121300,563,ABCG8,ENSG00000143921,1200
3342786,1200-Sleeplessness_insomnia,121300,563,THADA,ENSG00000115970,1200
3444215,1200-Sleeplessness_insomnia,600072,331,MEIS1,ENSG00000143995,1200


In [60]:
phewas_ukb_results = phewas_ukb_results.assign(multixcan_pval=np.nan)

for i, d in phewas_ukb_results.iterrows():
    gene_id = d['gene_id']
    if gene_id not in genes_associations.index:
        continue
    
    ukb_fullcode = d['trait']
    gene_trait_pval = genes_associations.loc[gene_id, ukb_fullcode]
    phewas_ukb_results.loc[i, 'multixcan_pval'] = gene_trait_pval

In [61]:
phewas_ukb_results.shape

(2204, 7)

In [62]:
phewas_ukb_results = phewas_ukb_results.dropna(subset=['multixcan_pval'])

In [63]:
phewas_ukb_results.shape

(2204, 7)

In [64]:
phewas_ukb_results.head()

,trait,pheno_mim,phecode,gene_name,gene_id,ukb_code,multixcan_pval
319791,1200-Sleeplessness_insomnia,168605,378,IKZF4,ENSG00000123411,1200,0.000826
319794,1200-Sleeplessness_insomnia,168605,378,PRIM1,ENSG00000198056,1200,0.231724
3342736,1200-Sleeplessness_insomnia,121300,563,ABCG8,ENSG00000143921,1200,0.136159
3342786,1200-Sleeplessness_insomnia,121300,563,THADA,ENSG00000115970,1200,0.100410
3444215,1200-Sleeplessness_insomnia,600072,331,MEIS1,ENSG00000143995,1200,0.000049


In [65]:
phewas_ukb_results.multixcan_pval.describe().apply(str)

count                    2204.0
mean        0.20251534090827306
std          0.2842019193881507
min                         0.0
25%      1.5066258736846786e-06
50%        0.027621561507205367
75%         0.34408382778924473
max          0.9982749103428394
Name: multixcan_pval, dtype: object

In [76]:
phewas_ukb_results[phewas_ukb_results['trait'] == '1200-Sleeplessness_insomnia'].shape

(48, 7)

### Count

In [66]:
total_phewas_catalog = phewas_ukb_results.shape[0]
display(total_phewas_catalog)

2204

In [67]:
# 0.05
hits = phewas_ukb_results[phewas_ukb_results['multixcan_pval'] < 0.05].shape[0]
display(hits)

1179

In [68]:
(hits / total_phewas_catalog) * 100.0

53.49364791288566

In [69]:
# 0.01
hits = phewas_ukb_results[phewas_ukb_results['multixcan_pval'] < 0.01].shape[0]
display(hits)

1005

In [70]:
(hits / total_phewas_catalog) * 100.0

45.5989110707804

In [71]:
SIGNIFICANT_PVALUE = 0.05 / total_phewas_catalog
display(SIGNIFICANT_PVALUE)

2.268602540834846e-05

In [72]:
# pvalue threshold
hits = phewas_ukb_results[phewas_ukb_results['multixcan_pval'] < SIGNIFICANT_PVALUE].shape[0]
display(hits)

664

In [73]:
(hits / total_phewas_catalog) * 100.0

30.127041742286753